In [1]:
pip install miditok miditoolkit

   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 2.1/2.1 MB 57.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 35.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!git clone https://github.com/music-x-lab/POP909-Dataset.git
!cd POP909-Dataset

Cloning into 'POP909-Dataset'...
Updating files:   7% (590/7450)
Updating files:   8% (596/7450)
Updating files:   9% (671/7450)
Updating files:  10% (745/7450)
Updating files:  11% (820/7450)
Updating files:  12% (894/7450)
Updating files:  13% (969/7450)
Updating files:  14% (1043/7450)
Updating files:  15% (1118/7450)
Updating files:  16% (1192/7450)
Updating files:  17% (1267/7450)
Updating files:  18% (1341/7450)
Updating files:  19% (1416/7450)
Updating files:  20% (1490/7450)
Updating files:  21% (1565/7450)
Updating files:  22% (1639/7450)
Updating files:  23% (1714/7450)
Updating files:  23% (1723/7450)
Updating files:  24% (1788/7450)
Updating files:  25% (1863/7450)
Updating files:  26% (1937/7450)
Updating files:  27% (2012/7450)
Updating files:  28% (2086/7450)
Updating files:  29% (2161/7450)
Updating files:  30% (2235/7450)
Updating files:  31% (2310/7450)
Updating files:  32% (2384/7450)
Updating files:  33% (2459/7450)
Updating files:  34% (2533/7450)
Updating files:  

In [1]:
import os
import pretty_midi   

def extract_notes_from_midi(file_path):
    midi = pretty_midi.PrettyMIDI(file_path)
    notes = []
    for instrument in midi.instruments:
        if not instrument.is_drum:
            for note in instrument.notes:
                pitch = note.pitch
                start = note.start
                duration = round(note.end - start, 3)
                notes.append((start, pitch, duration))
    notes.sort()           # sort by start-time
    return notes


def collect_pop909_notes():
    base_dir = r"C:\Documents\CompSci\CSE153\assignment2\POP909-Dataset\POP909"
    all_notes = {}

    # sub-folders 001 → 909 (zero-padded to width 3)
    for folder_id in range(1, 910):
        folder_name = f"{folder_id:03d}"
        folder_path = os.path.join(base_dir, folder_name)

        # skip if the folder somehow doesn’t exist
        if not os.path.isdir(folder_path):
            continue

        # any file ending in .mid or .midi
        for fname in os.listdir(folder_path):
            if fname.lower().endswith((".mid", ".midi")):
                fpath = os.path.join(folder_path, fname)
                all_notes[fpath] = extract_notes_from_midi(fpath)

    return all_notes

In [2]:
def quantize(value, step=0.25):
    # Round to nearest multiple of step
    return round(value / step) * step

def notes_to_token_sequence(notes, time_step=0.25):
    tokens = []
    prev_start = 0.0

    for start, pitch, duration in notes:
        time_shift = quantize(start - prev_start, time_step)
        token = f"TS_{time_shift:.2f}_P_{pitch}_D_{duration:.2f}"
        tokens.append(token)
        prev_start = start

    return tokens

In [3]:
notes_by_file = collect_pop909_notes()
print(f"Parsed {len(notes_by_file)} MIDI files.")
# e.g. inspect the first one
some_file, notes = next(iter(notes_by_file.items()))
print(some_file)
print(notes[:10])   # first 10 note tuples

Parsed 909 MIDI files.
C:\Documents\CompSci\CSE153\assignment2\POP909-Dataset\POP909\001\001.mid
[(2.3888829166666667, 66, 0.283), (2.7222154166666668, 47, 0.926), (2.7222154166666668, 75, 0.329), (2.888881666666667, 54, 0.79), (3.055547916666667, 59, 0.526), (3.055547916666667, 73, 0.182), (3.222214166666667, 66, 0.811), (3.388880416666667, 71, 0.206), (3.722212916666667, 80, 0.132), (3.888879166666667, 82, 0.16)]


In [4]:
token_sequence = notes_to_token_sequence(notes)
print(token_sequence[:100])

['TS_2.50_P_66_D_0.28', 'TS_0.25_P_47_D_0.93', 'TS_0.00_P_75_D_0.33', 'TS_0.25_P_54_D_0.79', 'TS_0.25_P_59_D_0.53', 'TS_0.00_P_73_D_0.18', 'TS_0.25_P_66_D_0.81', 'TS_0.25_P_71_D_0.21', 'TS_0.25_P_80_D_0.13', 'TS_0.25_P_82_D_0.16', 'TS_0.25_P_49_D_0.83', 'TS_0.00_P_80_D_0.83', 'TS_0.25_P_56_D_0.59', 'TS_0.25_P_61_D_0.61', 'TS_0.25_P_65_D_0.46', 'TS_0.50_P_66_D_0.23', 'TS_0.25_P_46_D_1.09', 'TS_0.00_P_75_D_0.34', 'TS_0.25_P_53_D_0.89', 'TS_0.25_P_58_D_0.98', 'TS_0.00_P_73_D_0.52', 'TS_0.25_P_61_D_0.74', 'TS_0.50_P_70_D_1.04', 'TS_0.25_P_51_D_0.35', 'TS_0.25_P_54_D_0.86', 'TS_0.25_P_58_D_0.80', 'TS_0.25_P_66_D_0.73', 'TS_0.50_P_73_D_0.20', 'TS_0.25_P_47_D_0.90', 'TS_0.00_P_78_D_0.17', 'TS_0.25_P_54_D_0.73', 'TS_0.00_P_80_D_0.08', 'TS_0.25_P_59_D_0.59', 'TS_0.00_P_82_D_0.58', 'TS_0.25_P_63_D_0.63', 'TS_0.50_P_73_D_0.14', 'TS_0.25_P_49_D_0.92', 'TS_0.00_P_56_D_0.23', 'TS_0.00_P_61_D_0.55', 'TS_0.00_P_78_D_0.14', 'TS_0.25_P_80_D_0.09', 'TS_0.25_P_56_D_0.21', 'TS_0.00_P_82_D_0.46', 'TS_0.25_P

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

# 1. Build vocab
unique_tokens = sorted(set(token_sequence))
token_to_id = {tok: i for i, tok in enumerate(unique_tokens)}
id_to_token = {i: tok for tok, i in token_to_id.items()}

# 2. Encode tokens to ids
encoded_sequence = [token_to_id[tok] for tok in token_sequence]

seq_length = 20  # how many tokens in input sequence

inputs = []
targets = []

for i in range(len(encoded_sequence) - seq_length):
    inputs.append(encoded_sequence[i:i+seq_length])
    targets.append(encoded_sequence[i+1:i+seq_length+1])

inputs = np.array(inputs)
targets = np.array(targets)

vocab_size = len(unique_tokens)
embedding_dim = 64
rnn_units = 128

model = tf.keras.Sequential([
    layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    layers.LSTM(rnn_units, return_sequences=True),
    layers.Dense(vocab_size, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

model.fit(inputs, targets, epochs=30, batch_size=64)


c:\Users\schng\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step - loss: 6.0607
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 5.6552
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 5.1324
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 4.5618
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 4.1047
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 3.6972
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.3318
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.9730
Epoch 9/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 2.7093
Epoch 10/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.5084
Epoch 11/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2.2942
Epoch 12/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 2.1215
Epoch 13/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 1.8951
Epoch 14/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 1.7773
Epoch 15/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 1.5671
Epoc

In [6]:
def generate_tokens(model, seed_sequence, gen_length=1000, temperature=1.0):
    generated = list(seed_sequence)
    for _ in range(gen_length):
        input_seq = np.array(generated[-seq_length:])[None, :]  # batch size 1
        preds = model.predict(input_seq)[0, -1]
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        next_id = np.random.choice(len(preds), p=preds)
        generated.append(next_id)
    return generated

# Start generation with the first sequence as seed
seed_seq = encoded_sequence[:seq_length]
generated_ids = generate_tokens(model, seed_seq)

generated_tokens = [id_to_token[i] for i in generated_ids]
print(generated_tokens)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━

In [7]:
def token_sequence_to_notes(token_sequence):
    notes = []
    current_time = 0.0

    for token in token_sequence:
        try:
            parts = token.split('_')
            step = float(parts[1])
            pitch = int(parts[3])
            duration = float(parts[5])
        except:
            continue  # skip malformed tokens

        start_time = current_time + step
        end_time = start_time + duration
        notes.append((pitch, start_time, end_time))
        current_time = start_time  # update time based on step

    return notes

def notes_to_midi_file(notes, output_file='generated.mid'):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)

    for pitch, start, end in notes:
        note = pretty_midi.Note(velocity=100, pitch=pitch, start=start, end=end)
        instrument.notes.append(note)

    midi.instruments.append(instrument)
    midi.write(output_file)
    print(f"✅ MIDI file saved as {output_file}")

notes = token_sequence_to_notes(generated_tokens)
notes_to_midi_file(notes, 'generated_music.mid')


✅ MIDI file saved as generated_music.mid
